# Image Classification with CNN

In order to train a machine learning model on a set of images, it's often necessary to ensure that all the images are of the same size. This is because most machine learning algorithms require a fixed input size, and working with images of different sizes can introduce challenges during training and inference.

In [1]:
#First we will set the same size of an each image
#We define the size of the input images, which will be 200x300 pixels.
import os
from PIL import Image

source_folder = 'Images/test/headnshoulders' 
destination_folder = 'Images/test/headnshoulders'
target_size = (200, 300)

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for filename in os.listdir(source_folder):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.webp') :
        img = Image.open(os.path.join(source_folder, filename))
        img = img.resize(target_size, resample=Image.BICUBIC)
        img.save(os.path.join(destination_folder, filename))


In [2]:
#importing the required libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# define the size of the input images
img_size = (200, 300)

# set the directories for the training and validation
train_dir = "Images/train"
val_dir = "Images/test"

# define the batch size and number of epochs for training
batch_size = 32
epochs = 10


In [4]:
# create a data generator for the training data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    classes=["8xShampoo", "saslic", "wow", "headnshoulders", "mamaearth"]
)

Found 40 images belonging to 5 classes.


In [5]:
# create a data generator for the validation data
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    classes=["8xShampoo", "saslic", "wow", "headnshoulders", "mamaearth"]
)


Found 29 images belonging to 5 classes.


In [6]:
# define the architecture of the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(5, activation="softmax")
])

In [7]:
# compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [8]:
# train the model on the training data and validate on the validation data
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/10
2/2 [==============================] - 7s 4s/step - loss: 9.0957 - accuracy: 0.1750 - val_loss: 5.3547 - val_accuracy: 0.3448
Epoch 2/10
2/2 [==============================] - 4s 3s/step - loss: 4.8732 - accuracy: 0.4000 - val_loss: 1.4648 - val_accuracy: 0.3448
Epoch 3/10
2/2 [==============================] - 4s 1s/step - loss: 1.4189 - accuracy: 0.6250 - val_loss: 1.5244 - val_accuracy: 0.3103
Epoch 4/10
2/2 [==============================] - 4s 3s/step - loss: 1.3779 - accuracy: 0.4500 - val_loss: 1.3374 - val_accuracy: 0.4828
Epoch 5/10
2/2 [==============================] - 4s 4s/step - loss: 0.8484 - accuracy: 0.6500 - val_loss: 1.0235 - val_accuracy: 0.5862
Epoch 6/10
2/2 [==============================] - 4s 2s/step - loss: 0.6422 - accuracy: 0.8750 - val_loss: 1.0106 - val_accuracy: 0.4828
Epoch 7/10
2/2 [==============================] - 4s 3s/step - loss: 0.6299 - accuracy: 0.7500 - val_loss: 1.0776 - val_accuracy: 0.5862
Epoch 8/10
2/2 [=========================

In [9]:
# evaluate the model on the testing data
test_loss, test_acc = model.evaluate(val_generator)

1/1 [==============================] - 1s 849ms/step - loss: 0.6327 - accuracy: 0.8966


In [10]:
# make predictions on new images using the trained model
new_image_path = "samples/13.jpg"
new_image = tf.keras.preprocessing.image.load_img(new_image_path, target_size=img_size)
new_image_array = tf.keras.preprocessing.image.img_to_array(new_image)
new_image_array = np.expand_dims(new_image_array, axis=0)
prediction = model.predict(new_image_array)

1/1 [==============================] - 0s 279ms/step


In [11]:
# get the index of the highest probability in the prediction array
predicted_class_index = np.argmax(prediction)

# get the class name using the index and the classes list from the data generator
classes_list = train_generator.class_indices
predicted_class = list(classes_list.keys())[list(classes_list.values()).index(predicted_class_index)]

# print the predicted class
print("Predicted class:", predicted_class)

Predicted class: mamaearth


# If given input is a folder with different images,then you can run the code below----------------------------->

In [12]:
import os
import cv2
# Loop through the images in the directory
for filename in os.listdir('samples'):
    # Load and preprocess the image
    img = cv2.imread(os.path.join('samples', filename))
    img = cv2.resize(img, (300, 200))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0

    # Predict the class probabilities for the image
    probs = model.predict(img)[0]
    classes=["8xShampoo", "saslic", "wow", "headnshoulders", "mamaearth"]
    # Get the predicted class label
    class_idx = np.argmax(probs)
    class_label = classes[class_idx]

    # Print the predicted class label
    print('Image:', filename, 'predicted class:', class_label)


1/1 [==============================] - 0s 78ms/step
Image: 13.jpg predicted class: saslic
1/1 [==============================] - 0s 71ms/step
Image: 19.jpg predicted class: wow
1/1 [==============================] - 0s 89ms/step
Image: 20.jpg predicted class: wow
1/1 [==============================] - 0s 67ms/step
Image: 21.jpg predicted class: wow
1/1 [==============================] - 0s 80ms/step
Image: 23.jpg predicted class: wow
1/1 [==============================] - 0s 92ms/step
Image: 24.jpg predicted class: wow
1/1 [==============================] - 0s 72ms/step
Image: 40144740_3-mamaearth-ubtan-face-wash.webp predicted class: wow
1/1 [==============================] - 0s 77ms/step
Image: 40197429_1-mamaearth-charcoal-face-scrub-with-charcoal-walnut-for-deep-exfoliation-for-oily-skin-normal-skin.webp predicted class: mamaearth
1/1 [==============================] - 0s 79ms/step
Image: 5.jpg predicted class: 8xShampoo
1/1 [==============================] - 0s 78ms/step
Image: 6.